In [39]:
import pandas as pd
import re

def read_data():
    df = pd.read_excel('./prep_data.xlsx',header = 0)
    df = df[df.columns[1:]]
    words = df['Word'].to_list()
    data = []

    for i in range(len(words)):
        for letter in re.findall('[a-z]',words[i]):
            data.append(letter)

    return words,data

def create_bigram(data):
    bigram_list = []
    bigram_cnt = {}
    unigram_cnt = {}
    for i in range(len(data) - 1):
        if i < len(data) - 1 and (i % 5 != 4):
            bigram_list.append( (data[i],data[i + 1]))

            if (data[i], data[i + 1]) in bigram_cnt:
                bigram_cnt[(data[i], data[i + 1])] += 1
            else:
                bigram_cnt[(data[i], data[i + 1])] = 1 
        
        if(data[i] in unigram_cnt):
            unigram_cnt[data[i]] += 1
        else:
            unigram_cnt[data[i]] = 1

    return bigram_list,bigram_cnt, unigram_cnt

def get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt):
    prob_list = {}
    for bigram in bigram_list:
        letter1 = bigram[0]
        letter2 = bigram[1]
        prob_list[bigram] = (bigram_cnt.get(bigram)) / (unigram_cnt.get(letter1))

    return prob_list

def get_word_prob(word,bigram_prob):
    letters = re.findall('[a-z]',word)
    word_bigram_list = []
    output_prob = 1

    for i in range(len(letters) - 1):
        if i < len(letters) - 1:
            word_bigram_list.append((letters[i], letters[i + 1]))

    for i in range(len(word_bigram_list)):
        if word_bigram_list[i] in bigram_prob:
            output_prob *= bigram_prob[word_bigram_list[i]]
        else:
            output_prob *= 0
    return output_prob
## Main Function
words, data = read_data()
bigram_list, bigram_cnt, unigram_cnt = create_bigram(data)

bigram_prob = get_bigram_prob(bigram_list, bigram_cnt, unigram_cnt)

sum = 0
for word in words:
    sum += get_word_prob(word,bigram_prob)
print(sum)

a = get_word_prob("eerie",bigram_prob)


0.028636887969402156
3.15461678165996e-05


In [38]:
print(bigram_prob)

{('s', 'l'): 0.058823529411764705, ('l', 'u'): 0.04504504504504504, ('u', 'm'): 0.06349206349206349, ('m', 'p'): 0.12727272727272726, ('c', 'r'): 0.08571428571428572, ('r', 'a'): 0.1076923076923077, ('a', 'n'): 0.08974358974358974, ('n', 'k'): 0.05813953488372093, ('g', 'o'): 0.09803921568627451, ('o', 'r'): 0.12307692307692308, ('r', 'g'): 0.03076923076923077, ('g', 'e'): 0.17647058823529413, ('q', 'u'): 1.0, ('u', 'e'): 0.047619047619047616, ('e', 'r'): 0.15384615384615385, ('r', 'y'): 0.06153846153846154, ('d', 'r'): 0.07547169811320754, ('r', 'i'): 0.1076923076923077, ('i', 'n'): 0.25742574257425743, ('a', 'b'): 0.01282051282051282, ('b', 'b'): 0.034482758620689655, ('b', 'e'): 0.27586206896551724, ('e', 'y'): 0.016483516483516484, ('t', 'a'): 0.0703125, ('n', 'g'): 0.11627906976744186, ('g', 'y'): 0.0784313725490196, ('p', 'a'): 0.07936507936507936, ('n', 'i'): 0.06976744186046512, ('i', 'c'): 0.07920792079207921, ('s', 'o'): 0.03529411764705882, ('o', 'l'): 0.046153846153846156, 